# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cityDataDF = pd.read_csv("output_data/cities.csv")
cityDataDF.head()

,City_ID,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country
0,0,Omboue,-1.5746,9.2618,76.80,broken clouds,90,59,6.26,GA
1,1,Iqaluit,63.7506,-68.5145,17.33,few clouds,78,20,0.00,CA
2,2,Saucillo,28.0167,-105.2833,69.03,clear sky,23,0,2.86,MX
3,3,Bluff,-46.6000,168.3333,54.48,overcast clouds,83,100,11.41,NZ
4,4,Sabang,5.8933,95.3214,80.78,overcast clouds,78,100,11.27,ID


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [6]:
locations = cityDataDF[["Lat", "Lng"]]
humidity = cityDataDF["Humidity"]
fig = gmaps.figure()
heatlayer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)
fig.add_layer(heatlayer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
weatherDF = cityDataDF.loc[
    (cityDataDF["Max Temp"] > 70) & (cityDataDF["Max Temp"] < 80) & (cityDataDF["Wind Speed"] < 10) & (cityDataDF["Cloudiness"] == 0)
].dropna()

weatherDF.head()

,City_ID,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country
24,24,Port Elizabeth,-33.9180,25.5701,70.32,clear sky,76,0,6.91,ZA
94,94,Poum,-20.2333,164.0167,78.10,clear sky,77,0,4.25,NC
112,112,Veraval,20.9000,70.3667,78.80,clear sky,55,0,6.26,IN
122,122,Araouane,18.9048,-3.5265,79.47,clear sky,13,0,7.56,ML
132,132,Fortuna,40.5982,-124.1573,71.92,clear sky,49,0,5.75,US


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = weatherDF[["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

C:\Users\joshu\AppData\Local\Temp\ipykernel_29780\4232746571.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Hotel Name
24,Port Elizabeth,ZA,-33.9180,25.5701,
94,Poum,NC,-20.2333,164.0167,
112,Veraval,IN,20.9000,70.3667,
122,Araouane,ML,18.9048,-3.5265,
132,Fortuna,US,40.5982,-124.1573,
146,Moctezuma,MX,29.8000,-109.6667,
156,Kibala,TD,9.1104,18.3463,
167,Saint-Pierre,RE,-21.3393,55.4781,
315,Abnub,EG,27.2696,31.1511,
319,Fayaoue,NC,-20.6500,166.5333,


In [14]:
param = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
    
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    param["location"] = f"{lat}, {lng}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    nameAddress = requests.get(url, params=param).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = nameAddress["results"][0]["name"]
        
    except:
        print(f"Error finding lodging in {row['City']}. Skip.")

Error finding lodging in Poum. Skip.
Error finding lodging in Araouane. Skip.
Error finding lodging in Agadez. Skip.


In [15]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
24,Port Elizabeth,ZA,-33.9180,25.5701,39 On Nile Guest House
94,Poum,NC,-20.2333,164.0167,
112,Veraval,IN,20.9000,70.3667,Lords Inn Somnath
122,Araouane,ML,18.9048,-3.5265,
132,Fortuna,US,40.5982,-124.1573,Super 8 by Wyndham Fortuna
146,Moctezuma,MX,29.8000,-109.6667,Hotel Lobohombo
156,Kibala,TD,9.1104,18.3463,Prasant Hotels
167,Saint-Pierre,RE,-21.3393,55.4781,Alize Plage
315,Abnub,EG,27.2696,31.1511,دوار ال عبد الجواد شرق البلد بابنوب
319,Fayaoue,NC,-20.6500,166.5333,Chez Dydyce Guesthouse


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))